In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
def calculate_entropy(df: pd.DataFrame, swappable_shifts: list[str]) -> list[int]:
    entropy = []

    for worker in df.columns:
        curr_entropy = 0
        for shift in swappable_shifts:
            

In [2]:
def get_swap_scores(df: pd.DataFrame, swappable_shifts: list[str]) -> dict:
    """Calculate score of current dataframe."""
    scores = {}
    
    for person in df.columns:
        values = df[person].dropna().tolist() 
        shifts_to_check = [item for item in values if item in swappable_shifts]
        if not values:
            scores[person] = 0
            continue
          
        score = 0
        letter_counts = Counter(shifts_to_check)
        print("letter_counts")
        print(letter_counts)
        
        for letter, count in letter_counts.items():
            letter_positions = {}
            if count > 1:
                letter_positions[letter] = []
                for i, value in enumerate(df[person]):
                    if value == letter:
                        letter_positions[value].append(i)
                    print(letter_positions)

                for letter, positions in letter_positions.items():
                        for i in range(len(positions)):
                            for j in range(i + 1, len(positions)):
                                distance = positions[j] - positions[i]
                                inverse_distance = 1.0 / distance
                                score += inverse_distance
                
        scores[person] = score
    
    return scores

In [6]:
data = {'Tech': ['A', None,'C'  , 'D' , 'A', None, 'B'],
        'Zack': ['B', None, None, None, 'D', 'D'  , 'A'],
        'Dani': ['C', None, None, None, None, None, 'D']}

df = pd.DataFrame(data)
worker_to_assign = df.isna().sum().sort_values(ascending=False).index[0]

In [11]:
df.isna().sum().sort_values(ascending=False).index.tolist()

['Dani', 'Zack', 'Tech']

In [13]:
df.isna().sum(axis=1).sort_values(ascending=False).index.tolist()

[1, 2, 3, 5, 4, 0, 6]

In [5]:
swappable_shifts = ['A','B','C','D']

print(get_swap_scores(df,swappable_shifts))

letter_counts
Counter({'A': 2, 'C': 1, 'D': 1, 'B': 1})
{'A': [0]}
{'A': [0]}
{'A': [0]}
{'A': [0]}
{'A': [0, 4]}
{'A': [0, 4]}
{'A': [0, 4]}
letter_counts
Counter({'D': 2, 'B': 1, 'A': 1})
{'D': []}
{'D': []}
{'D': []}
{'D': []}
{'D': [4]}
{'D': [4, 5]}
{'D': [4, 5]}
letter_counts
Counter({'C': 1, 'D': 1})
{'Tech': 0.25, 'Zack': 1.0, 'Dani': 0}


In [85]:
swappable_shifts = ['A','C','D']

swap_scores = get_swap_scores(df, swappable_shifts)
sorted_scores = sorted(swap_scores.items(), key=lambda item: item[1], reverse=True)

initial_score = sum(swap_scores.values())
current_best_score = initial_score

#print(sorted_scores)
#print(sum(swap_scores.values()))

print(df)
print(f"initial score: {initial_score}")


for i,person in enumerate(sorted_scores):
    for row in df.index:
        #print(df.at[row,person[0]], i)
        for j in range(i+1, len(sorted_scores)):
            main_person = sorted_scores[i][0]
            potential_swapper = sorted_scores[j][0]

            swapped_df = df.copy()
            temp = swapped_df.at[row,main_person]
            swapped_df.at[row,main_person] = swapped_df.at[row,potential_swapper]
            swapped_df.at[row,potential_swapper] = temp

            new_scores = get_swap_scores(swapped_df, swappable_shifts)
            new_sum = sum(new_scores.values())

            #print(f"{row}: {main_person} vs {potential_swapper}: {new_sum}")


            if new_sum < current_best_score:
                #print(f"new better score! {new_sum}")
                #print(df)
                current_best_score = new_sum
                df = swapped_df.copy()
                continue        

print(df)
score = sum(get_swap_scores(df,swappable_shifts).values())
print(f"final score: {score}")

   Tech  Zack Daniel
0     A     B      C
1  None  None   None
2     C  None   None
3     D  None   None
4     A     D   None
5  None     D   None
6     B     A      D
initial score: 1.25
   Tech  Zack Daniel
0     B     C      A
1  None  None   None
2     C  None   None
3     D  None   None
4     A  None      D
5  None     D   None
6     D     A      B
final score: 0.3333333333333333
